# Symbolic Partial Derivative Routine

## Authors: Zach Etienne & Tyler Knowles

## This module contains a routine for computing an analytic partial derivative of a mathematical expression.

**Notebook Status:** <font color='red'><b> In progress </b></font>

**Validation Notes:** The module has been validated by comparing results to finite-difference derivative values in LALSuite.  <font color='red'><b> Add more info later </b></font>

<a id='intro'></a>

## Introduction
$$\label{intro}$$

Motivated by Mathematica being unable to generate readable partial derivatives of the SEOBNRv3 Hamiltonian, we wrote our own partial derivatve routine.

In [1]:
# Partial derivative routine
# Import necessary modules

import os,sys                    # Standard Python modules for multiplatform OS-level functions
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

from outputC import * #Check what is imported and remove *
import sympy as sp

bigstring = """
sigmaKerr0 = s1x + s2x
sigmaKerr1 = s1y + s2y
sigmaKerr2 = s1z + s2z
quagsire = eta*eta
s1dots1 = s1x*s1x + s1y*s1y + s1z*s1z + quagsire
s2dots2 = s2x*s2x + s2y*s2y + s2z*s2z
r2 = x*x + y*y + z*z
r = sp.sqrt(r2)
u = 1/r
tmppx = px - r
tmppy = py - r
tmppz = pz - r
"""

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1:](#step1) Split left- and right-hand sides of expressions
1. [Step 2:](#step2) Convert left-hand sides to function notation
1. [Step 3:](#step3) Produce list of free variables
1. [Step 4:](#step4) Declare input constants
1. [Step 5:](#step5) Convert free variables to function notation
1. [Step 6:](#step6) Differentiate!
1. [Step 7:](#step7) Simplify derivative expressions
1. [Step 8:](#step8) Differentiate with respect to a specific free variable
1. [Step 9:](#step9) Compute derivatives with respect to each free variable
1. [Step 10:](#step10) Output result
1. [Step 11:](#latex_pdf_output) Output this notebook to $\LaTeX$-formatted PDF file

<a id='step1'></a>

# Step 1: Split lift- and right-hand sides of expressions \[Back to [top](#toc)\]
$$\label{step1}$$

We take the input string and split it first by line (carriage return), then by "=".  Doing so allows us to manipulate the right- and left-hand sides appropriately.  We store the left- and right-hand sides in the array $\texttt{lr}$, which consists of $\texttt{lhrh}$ arrays with left-hand sides $\texttt{lhs}$ and right-hand sides $\texttt{rhs}$.  Note that $\texttt{Lambda}$ is a protected keyword in Python, so the variable $\Lambda$ in the Hamiltonian is renamed $\texttt{Lamb}$.

In [2]:
# Split bigstring by carriage returns:
string_lines = bigstring.splitlines()

# Create "lr" array, which will store each left-hand side and right-hand side as strings.
lr = []
# Loop over each line in bigstring
for i in range(len(string_lines)):
    # Ignore lines with 2 or fewer characters and those starting with #
    if len(string_lines[i]) > 2 and string_lines[i][0] != "#":
        # Split each line by its equals sign.
        split_line = string_lines[i].split("=")
        # Append to the "lr" array, removing spaces, "sp." prefixes, and replacing Lambda->Lamb
        #       (Lambda is a protected keyword):
        lr.append(lhrh(lhs=split_line[0].replace(" ","").replace("Lambda","Lamb"),
                       rhs=split_line[1].replace(" ","").replace("sp.","").replace("Lambda","Lamb")))

[lhrh(lhs='sigmaKerr0', rhs='s1x+s2x'), lhrh(lhs='sigmaKerr1', rhs='s1y+s2y'), lhrh(lhs='sigmaKerr2', rhs='s1z+s2z'), lhrh(lhs='quagsire', rhs='eta*eta'), lhrh(lhs='s1dots1', rhs='s1x*s1x+s1y*s1y+s1z*s1z+quagsire'), lhrh(lhs='s2dots2', rhs='s2x*s2x+s2y*s2y+s2z*s2z'), lhrh(lhs='r2', rhs='x*x+y*y+z*z'), lhrh(lhs='r', rhs='sqrt(r2)'), lhrh(lhs='u', rhs='1/r'), lhrh(lhs='tmppx', rhs='px-r'), lhrh(lhs='tmppy', rhs='py-r'), lhrh(lhs='tmppz', rhs='pz-r')]


<a id='step2'></a>

# Step 2: Convert left-hand sides to function notation \[Back to [top](#toc)\]
$$\label{step2}$$

In order to compute the partial derivative of each expression, we mark each variable (left-hand side) as a function with argument $\texttt{xx}$.  We simultaneously split right- and left-hand sides into their own arrays, $\texttt{fhss}$ and $\texttt{lhss}$, respectively.

In [3]:
xx = sp.Symbol('xx')
func = []
lhss = []
rhss = []
for i in range(len(lr)):
    func.append(sp.sympify(sp.Function(lr[i].lhs)(xx)))
    lhss.append(sp.sympify(lr[i].lhs))
    rhss.append(sp.sympify(lr[i].rhs))

[sigmaKerr0(xx), sigmaKerr1(xx), sigmaKerr2(xx), quagsire(xx), s1dots1(xx), s2dots2(xx), r2(xx), r(xx), u(xx), tmppx(xx), tmppy(xx), tmppz(xx)]
[sigmaKerr0, sigmaKerr1, sigmaKerr2, quagsire, s1dots1, s2dots2, r2, r, u, tmppx, tmppy, tmppz]
[s1x + s2x, s1y + s2y, s1z + s2z, eta**2, quagsire + s1x**2 + s1y**2 + s1z**2, s2x**2 + s2y**2 + s2z**2, x**2 + y**2 + z**2, sqrt(r2), 1/r, px - r, py - r, pz - r]


<a id='step3'></a>

# Step 3: Produce list of free variables \[Back to [top](#toc)\]
$$\label{step3}$$

By ''free variables'' we mean the symbolic variables names in the right-hand sides.  We first create a list of all such terms, including duplicates, and then strip the duplicates.

In [4]:
# Next get a list of all the "free symbols" in the RHS expressions.
full_symbol_list_with_dups = []
for i in range(len(lr)):
    for var in rhss[i].free_symbols:
        full_symbol_list_with_dups.append(var)

full_symbol_list = superfast_uniq(full_symbol_list_with_dups)

[s1x, s2x, s2y, s1y, s1z, s2z, eta, s1z, s1x, s1y, quagsire, s2y, s2x, s2z, y, z, x, r2, r, px, r, r, py, pz, r]
[s1x, s2x, s2y, s1y, s1z, s2z, eta, quagsire, y, z, x, r2, r, px, py, pz]


<a id='step4'></a>

# Step 4: Declare input constants \[Back to [top](#toc)\]
$$\label{step4}$$

We declare the constant values; derivatives with respect to these variables will be set to zero.

In [5]:
# Declare input constants:
m1,m2,eta,tortoise,dSO,dSS = sp.symbols("m1 m2 eta tortoise dSO dSS",real=True)
input_constants = [m1,m2,eta,tortoise,dSO,dSS]

<a id='step5'></a>

# Step 5: Convert free variables to function notation \[Back to [top](#toc)\]
$$\label{step5}$$

We compare the input constants to free variables and remove constatns from the free variable list.  We then mark the free varibles as functions with argument \texttt{xx}.

In [6]:
# Remove input constants from symbol list
for inputconst in input_constants:
    for symbol in full_symbol_list:
        if str(symbol) == str(inputconst):
            full_symbol_list.remove(symbol)

# Mark each free variable as a function with argument xx
full_function_list = []
for symb in full_symbol_list:
    func = sp.sympify(sp.Function(str(symb))(xx))
    full_function_list.append(func)
    for i in range(len(rhss)):
        for var in rhss[i].free_symbols:
            if str(var) == str(symb):
                rhss[i] = rhss[i].subs(var,func)

[s1x(xx), s2x(xx), s2y(xx), s1y(xx), s1z(xx), s2z(xx), quagsire(xx), y(xx), z(xx), x(xx), r2(xx), r(xx), px(xx), py(xx), pz(xx)]


<a id='step6'></a>

# Step 6: Differentiate! \[Back to [top](#toc)\]
$$\label{step6}$$

Now we differentiate the right-hand expressions!  We use the SymPy $\texttt{diff}$ command, differentiating with respect to $\texttt{xx}$.  After so doing, we remove $\texttt{(xx)}$ and "Derivative" (which is output by $\texttt{diff}$, and use "prm" suffix to denote the derivative with respect to $\texttt{xx}$.

In [7]:
lhss_deriv = []
rhss_deriv = []
for i in range(len(rhss)):
    lhss_deriv.append(sp.sympify(str(lhss[i])+"prm"))
    newrhs = sp.sympify(str(sp.diff(rhss[i],xx)).replace("(xx)","").replace(", xx","prm").replace("Derivative",""))
    rhss_deriv.append(newrhs)

[sigmaKerr0prm, sigmaKerr1prm, sigmaKerr2prm, quagsireprm, s1dots1prm, s2dots2prm, r2prm, rprm, uprm, tmppxprm, tmppyprm, tmppzprm]
[s1xprm + s2xprm, s1yprm + s2yprm, s1zprm + s2zprm, 0, quagsireprm + 2*s1x*s1xprm + 2*s1y*s1yprm + 2*s1z*s1zprm, 2*s2x*s2xprm + 2*s2y*s2yprm + 2*s2z*s2zprm, 2*x*xprm + 2*y*yprm + 2*z*zprm, r2prm/(2*sqrt(r2)), -rprm/r**2, pxprm - rprm, pyprm - rprm, pzprm - rprm]


<a id='step7'></a>

# Step 7: Simplify derivative expressions \[Back to [top](#toc)\]
$$\label{step7}$$

We declare a function to simply the derivative expressions.  In particular, we want to remove terms equal to zero.

In [8]:
# Derivative simplification function
def simplify_deriv(lhss_deriv,rhss_deriv):
    # Copy expressions into another array
    lhss_deriv_simp = []
    rhss_deriv_simp = []
    for i in range(len(rhss_deriv)):
        lhss_deriv_simp.append(lhss_deriv[i])
        rhss_deriv_simp.append(rhss_deriv[i])
    # If a right-hand side is 0, substitute value 0 for the corresponding left-hand side in later terms
    for i in range(len(rhss_deriv_simp)):
        if rhss_deriv_simp[i] == 0:
            for j in range(i+1,len(rhss_deriv_simp)):
                for var in rhss_deriv_simp[j].free_symbols:
                    if str(var) == str(lhss_deriv_simp[i]):
                        rhss_deriv_simp[j] = rhss_deriv_simp[j].subs(var,0)
    zero_elements_to_remove = []
    # Create array of indices for expressions that are zero
    for i in range(len(rhss_deriv_simp)):
        if rhss_deriv_simp[i] == sp.sympify(0):
            zero_elements_to_remove.append(i)

    # When removing terms that are zero, we need to take into account their new index (after each removal)
    count = 0
    for i in range(len(zero_elements_to_remove)):
        del lhss_deriv_simp[zero_elements_to_remove[i]+count]
        del rhss_deriv_simp[zero_elements_to_remove[i]+count]
        count -= 1
    return lhss_deriv_simp,rhss_deriv_simp

# Call the simplication function and then copy results
lhss_deriv_simp,rhss_deriv_simp = simplify_deriv(lhss_deriv,rhss_deriv)
lhss_deriv = lhss_deriv_simp
rhss_deriv = rhss_deriv_simp

<a id='step8'></a>

# Step 8: Differentiate with respect to a specific free variable \[Back to [top](#toc)\]
$$\label{step8}$$

In [Step 6](#step6) we took a generic derivative of each term, assuming it is a function of the varible $\textrm{xx}$.  We now define a function that will select a specific free variable for differentiation.

In [10]:
def deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,
                 s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0):

    # Copy expressions into another array
    lhss_deriv_new = []
    rhss_deriv_new = []
    for i in range(len(rhss_deriv)):
        lhss_deriv_new.append(lhss_deriv[i])
        rhss_deriv_new.append(rhss_deriv[i])
    # For each free symbol, replace it with the desired derivative
    for i in range(len(rhss_deriv_new)):
        for var in rhss_deriv_new[i].free_symbols:
            if str(var)=="xprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,xprm)
            elif str(var)=="yprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,yprm)
            elif str(var)=="zprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,zprm)
            elif str(var)=="pxprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,pxprm)
            elif str(var)=="pyprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,pyprm)
            elif str(var)=="pzprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,pzprm)
            elif str(var)=="s1xprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,s1xprm)
            elif str(var)=="s1yprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,s1yprm)
            elif str(var)=="s1zprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,s1zprm)
            elif str(var)=="s2xprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,s2xprm)
            elif str(var)=="s2yprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,s2yprm)
            elif str(var)=="s2zprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,s2zprm)
    # Simplify derivative expressions again
    lhss_deriv_simp,rhss_deriv_simp = simplify_deriv(lhss_deriv_new,rhss_deriv_new)
    return lhss_deriv_simp,rhss_deriv_simp

<a id='step9'></a>

# Step 9: Compute derivatives with respect to each free variable \[Back to [top](#toc)\]
$$\label{step9}$$

This needs to be made into a loop!

In [11]:
lhss_deriv_x,rhss_deriv_x = deriv_onevar(lhss_deriv,rhss_deriv, xprm=1,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,
                                         s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_y,rhss_deriv_y = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=1,zprm=0,pxprm=0,pyprm=0,pzprm=0,
                                         s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_z,rhss_deriv_z = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=1,pxprm=0,pyprm=0,pzprm=0,
                                         s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_px,rhss_deriv_px = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=0,pxprm=1,pyprm=0,pzprm=0,
                                           s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_py,rhss_deriv_py = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=1,pzprm=0,
                                           s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_pz,rhss_deriv_pz = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=1,pzprm=1,
                                           s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_s1x,rhss_deriv_s1x = deriv_onevar(lhss_deriv,rhss_deriv, xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,
                                             s1xprm=1,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_s1y,rhss_deriv_s1y = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,
                                             s1xprm=0,s1yprm=1,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_s1z,rhss_deriv_s1z = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,
                                             s1xprm=0,s1yprm=0,s1zprm=1,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_s2x,rhss_deriv_s2x = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,
                                             s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=1,s2yprm=0,s2zprm=0)
lhss_deriv_s2y,rhss_deriv_s2y = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,
                                             s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=1,s2zprm=0)
lhss_deriv_s2z,rhss_deriv_s2z = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,
                                             s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=1)

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 10]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[1, 2, 4, 5, 6, 7, 8, 9, 10]
[0, 2, 4, 5, 6, 7, 8, 9, 10]
[0, 1, 4, 5, 6, 7, 8, 9, 10]
[1, 2, 3, 5, 6, 7, 8, 9, 10]
[0, 2, 3, 5, 6, 7, 8, 9, 10]
[0, 1, 3, 5, 6, 7, 8, 9, 10]


<a id='step10'></a>

# Step 10: Output result \[Back to [top](#toc)\]
$$\label{step10}$$

We write the resulting derivatives in C code.

In [19]:
outstring = "/* SEOBNR Hamiltonian expression: */\n"
outstringsp = ""
outsplhs = []
outsprhs = []
for i in range(len(lr)):
    outstring += outputC(sp.sympify(lr[i].rhs),lr[i].lhs,"returnstring","outCverbose=False,includebraces=False,CSE_enable=False")
    outstringsp += lr[i].lhs+" = "+lr[i].rhs+"\n"
    outsplhs.append(sp.sympify(lr[i].lhs))
    outsprhs.append(sp.sympify(lr[i].rhs))
outstring += "\n\n\n/* SEOBNR \partial_x H expression: */\n"
for i in range(len(lhss_deriv_x)):
    outstring += outputC(rhss_deriv_x[i],str(lhss_deriv_x[i]),"returnstring","outCverbose=False,includebraces=False,CSE_enable=False")
    outstringsp += str(lhss_deriv_x[i])+" = "+str(rhss_deriv_x[i])+"\n"
    outsplhs.append(lhss_deriv_x[i])
    outsprhs.append(rhss_deriv_x[i])

[sigmaKerr0, sigmaKerr1, sigmaKerr2, quagsire, s1dots1, s2dots2, r2, r, u, tmppx, tmppy, tmppz, r2prm, rprm, uprm, tmppxprm, tmppyprm, tmppzprm]
[s1x + s2x, s1y + s2y, s1z + s2z, eta**2, quagsire + s1x**2 + s1y**2 + s1z**2, s2x**2 + s2y**2 + s2z**2, x**2 + y**2 + z**2, sqrt(r2), 1/r, px - r, py - r, pz - r, 2*x, r2prm/(2*sqrt(r2)), -rprm/r**2, -rprm, -rprm, -rprm]


<a id='latex_pdf_output'></a>

# Step 11: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-SEOBNR_Derivative_Routine.pdf](Tutorial-SEOBNR_Derivative_Routine.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [15]:
import os,sys                    # Standard Python modules for multiplatform OS-level functions
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("NRPyPN_shortcuts",location_of_template_file=os.path.join(".."))

Notebook output to PDF is only supported on Linux systems, with pdflatex installed.
